In [16]:
import numpy as np
import surprise
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from data_helpers import *

#Load train and testset for the surprise models
file_path = "../data/data_surprise.csv"
trainset, testset = build_surprise_data(file_path)

#Loads ratings to predict
INPUT_PATH = "../data/sample_submission.csv"
ids = read_csv_sample(INPUT_PATH)

In [18]:
#Load train and testset for the custom models
#train, test = split_data(load_data("../data/data_train.csv"), p_test = 0.1)

number of items: 10000, number of users: 1000


In [19]:
from implementations import *

In [20]:
Xtest = []
Xids = []

#kNN Baseline Movie
rmse, Xtest, Xids, preds_test, predictions = knn_baseline_movie(trainset, testset, ids, Xtest, Xids)

kNN Baseline Movie
   Training RMSE:  0.6241475383015472
   Test RMSE:  0.9908785231084433
kNN Baseline User
   Training RMSE:  0.6641531098089988
   Test RMSE:  0.9947164198559775
SVD
   Training RMSE:  0.9543999695353427
   Test RMSE:  1.0019284327486202


In [37]:
from data_helpers import create_csv_submission

OUTPUT_PATH = "../data/submission.csv"
create_csv_submission(ids, predictions, OUTPUT_PATH)
print("File submission.csv ready to be submitted !")

File submission.csv ready to be submitted !
